In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures
from multiprocessing import Pool

# 【方案一】Wiki系列API

## wikipedia
- wikipedia | https://pypi.org/project/wikipedia/ 

In [7]:
import wikipedia
wikipedia.set_lang("en")
wikipedia.summary("Bottle")
wikipedia.summary("Bottle",sentences=1)
wikipedia.search("Bottle") # 检索Barack

'A bottle is a narrow-necked container made of an impermeable material (clay, glass, plastic, aluminium, etc.) in various shapes and sizes  to store and transport liquids (water, milk, beer, wine, ink, cooking oil, medicine, soft drinks, shampoo, and chemicals, etc.) and whose mouth at the bottling line can be sealed with an internal stopper, an external bottle cap, a closure, or a conductive "inner seal" using induction sealing.\nSome of the earliest bottles appeared in China, Phoenicia, Crete, and Rome.'

'A bottle is a narrow-necked container made of an impermeable material (clay, glass, plastic, aluminium, etc.) in various shapes and sizes  to store and transport liquids (water, milk, beer, wine, ink, cooking oil, medicine, soft drinks, shampoo, and chemicals, etc.) and whose mouth at the bottling line can be sealed with an internal stopper, an external bottle cap, a closure, or a conductive "inner seal" using induction sealing.'

['Bottle',
 'Bottle It In',
 'Coca-Cola',
 'Klein bottle',
 'Polyethylene terephthalate',
 'Bottle Rocket',
 'The Bottle',
 'Wine bottle',
 'Bottle episode',
 'Calabash']

In [42]:
try:
    ny = wikipedia.page("China") # 直接获取页面
    ny.title
    ny.url
    len(ny.content)
    len(ny.links)
    ny.links[0]
except wikipedia.DisambiguationError as e:
    print("目标词条有歧义")
except wikipedia.WikipediaException as e:
    print("未找到目标词条")

'China'

'https://en.wikipedia.org/wiki/China'

98415

1963

'+86'

## wikipediaapi
- wikipedia-API | https://pypi.org/project/Wikipedia-API/

In [33]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Python_(programming_language)')

# page_py.exists() # 检查页面是否存在
# page_py.title
# page_py.summary # 获取summary
# page_py.sections # 获取sections
# page_py.sections[0].sections # 获取二级section
# page_py.canonicalurl # 获得页面的url | 区别未知
# page_py.fullurl # 获得页面的url | 区别未知
# page_py.text # 内容
# page_py_an = page_py.langlinks['an'] # 获取当前页面其他语言形式
cats = page_py.categories # 获取category
[i for i in cats.values()]

# # 内容（使用html形式）
# wiki_html=wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.HTML)
# wiki_html.page("India").text

# 获取页面中所有连接（指向其他wiki页面的链接）
# links = page_py.links
# print(links.values())
# for title in sorted(links.keys()):
#     print(title,links[title])

[Category:All articles containing potentially dated statements (id: ??, ns: 14),
 Category:Articles containing potentially dated statements from August 2016 (id: ??, ns: 14),
 Category:Articles containing potentially dated statements from December 2018 (id: ??, ns: 14),
 Category:Articles containing potentially dated statements from March 2018 (id: ??, ns: 14),
 Category:Articles with Curlie links (id: ??, ns: 14),
 Category:Articles with short description (id: ??, ns: 14),
 Category:Class-based programming languages (id: ??, ns: 14),
 Category:Computational notebook (id: ??, ns: 14),
 Category:Computer science in the Netherlands (id: ??, ns: 14),
 Category:Cross-platform free software (id: ??, ns: 14),
 Category:Cross-platform software (id: ??, ns: 14),
 Category:Dutch inventions (id: ??, ns: 14),
 Category:Dynamically typed programming languages (id: ??, ns: 14),
 Category:Educational programming languages (id: ??, ns: 14),
 Category:Good articles (id: ??, ns: 14),
 Category:High-lev

KeyError: 'query'

### 获取category下所有的页面

In [163]:
# 获取category在的所有页面
def print_categorymembers(categorymembers, level=0, max_level=5):
        for c in categorymembers.values():
            if c.ns==14:
                print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
            if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
                print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)

cat = wiki_wiki.page("Category:Economy of Goa")
print("Category members: Category:Physics")
print_categorymembers(cat.categorymembers)

Category members: Category:Physics
*: Category:Businesspeople from Goa (ns: 14)
**: Category:Businesswomen from Goa (ns: 14)
*: Category:Communications in Goa (ns: 14)
**: Category:Media in Goa (ns: 14)
***: Category:International Film Festival of India (ns: 14)
***: Category:Journalists from Goa (ns: 14)
***: Category:Radio stations in Goa (ns: 14)
***: Category:Television stations in Goa (ns: 14)
*: Category:Companies based in Goa (ns: 14)
*: Category:Dams in Goa (ns: 14)
*: Category:Transport in Margao (ns: 14)
*: Category:Transport in Panaji (ns: 14)
*: Category:Tourism in Goa (ns: 14)
**: Category:Hotels in Goa (ns: 14)
**: Category:Tourist attractions in Goa (ns: 14)
***: Category:Tourist attractions in North Goa district (ns: 14)
****: Category:Beaches of North Goa district (ns: 14)
****: Category:Chorão (Island) (ns: 14)
*****: Category:Churches in Chorão Island (ns: 14)
******: Category:Roman Catholic churches in Chorão Island (ns: 14)
*****: Category:People of Chorão (Island)

### 获取category下的所有子孙category

In [153]:
# targetCategory = "History of metallurgy in the Indian subcontinent"
targetCategory = "China"
targetCategory = "Economy of Goa"

cat = wiki_wiki.page(f"Category:{targetCategory}")
sub_cat = [i.title for i in cat.categorymembers.values() if i.ns == 14]
portal = [i.title for i in cat.categorymembers.values() if i.ns == 100]

In [155]:
from treelib import Node, Tree
tree = Tree()
rootNode = tree.create_node(cat.title)
for i in cat.categorymembers.values():
     if i.ns == 14:
        _ = tree.create_node(i.title.split(":")[1],parent=rootNode.identifier)
        
tree.show()

Category:Economy of Goa
├── Businesspeople from Goa
├── Communications in Goa
├── Companies based in Goa
├── Dams in Goa
├── Tourism in Goa
├── Transport in Goa
├── Transport in Margao
└── Transport in Panaji



### 获取树结构

In [32]:
def putSubInTree(tree,current_node,cur_cat,max_level=3,init_level=0):
    for sub_cat in cur_cat.categorymembers.values():
        if sub_cat.ns == 14:
            sub_node = tree.create_node(sub_cat.title.split(":")[1],parent=current_node.identifier)
            if init_level < max_level:
                init_level = init_level + 1
                putSubInTree(tree,sub_node,sub_cat,max_level,init_level)
    return None

tree = Tree()
targetCategory = urllib.parse.quote("Hospitals in india")
targetCategory = re.sub("india","India","Economic_history_of_india")
cat = wiki_wiki.page(f"Category:{targetCategory}")
# cat = wiki_wiki.page("Category:Hospitals_in_India")
cat.exists()

currentNode = tree.create_node(cat.title.split(":")[1])
putSubInTree(tree,currentNode,cat,3)
tree.show()

','.join([tree[node].tag for node in tree.expand_tree(mode=Tree.DEPTH)])
','.join([tree[node].tag for node in tree.expand_tree(mode=Tree.WIDTH)])
','.join([tree[node].tag for node in tree.expand_tree(mode=Tree.ZIGZAG)])

True

Economic history of India
├── Child labour in India
├── Colonial Indian companies
│   ├── British East India Company
│   │   ├── British East India Company people
│   │   ├── Medals of the Honourable East India Company
│   │   ├── Military history of the British East India Company
│   │   ├── Properties of the East India Company
│   │   ├── Ships of the British East India Company
│   │   └── Treaties of the British East India Company
│   ├── Dutch East India Company
│   ├── French East India Company
│   └── Swedish East India Company
├── Customary units in India
├── Defunct companies of India
├── Defunct trade unions of India
├── Diamonds originating in India
├── Economic history of India (1947–present)
├── Economic history of Karnataka
├── Economic history of Tamil Nadu
├── Economy of British India
├── Financial history of India
├── Five-year plans of India
├── Goods manufactured in India
├── Historical currencies of India
├── History of agriculture in India
├── History of companies o

'Economic history of India,Child labour in India,Colonial Indian companies,British East India Company,British East India Company people,Medals of the Honourable East India Company,Military history of the British East India Company,Properties of the East India Company,Ships of the British East India Company,Treaties of the British East India Company,Dutch East India Company,French East India Company,Swedish East India Company,Customary units in India,Defunct companies of India,Defunct trade unions of India,Diamonds originating in India,Economic history of India (1947–present),Economic history of Karnataka,Economic history of Tamil Nadu,Economy of British India,Financial history of India,Five-year plans of India,Goods manufactured in India,Historical currencies of India,History of agriculture in India,History of companies of India,History of forestry in India,History of taxation in India,Indian inventions,Industrial history of India,Nationalisation in India,Pepper trade,Portuguese India 

'Economic history of India,Child labour in India,Colonial Indian companies,Customary units in India,Defunct companies of India,Defunct trade unions of India,Diamonds originating in India,Economic history of India (1947–present),Economic history of Karnataka,Economic history of Tamil Nadu,Economy of British India,Financial history of India,Five-year plans of India,Goods manufactured in India,Historical currencies of India,History of agriculture in India,History of companies of India,History of forestry in India,History of taxation in India,Indian inventions,Industrial history of India,Nationalisation in India,Pepper trade,Portuguese India Armadas,Trading posts in India,British East India Company,Dutch East India Company,French East India Company,Swedish East India Company,British East India Company people,Medals of the Honourable East India Company,Military history of the British East India Company,Properties of the East India Company,Ships of the British East India Company,Treaties of 

'Economic history of India,Trading posts in India,Portuguese India Armadas,Pepper trade,Nationalisation in India,Industrial history of India,Indian inventions,History of taxation in India,History of forestry in India,History of agriculture in India,Goods manufactured in India,Five-year plans of India,Financial history of India,Economy of British India,Economic history of Tamil Nadu,Economic history of Karnataka,Economic history of India (1947–present),Diamonds originating in India,Defunct trade unions of India,Defunct companies of India,Customary units in India,Historical currencies of India,History of companies of India,Colonial Indian companies,Child labour in India,British East India Company,Dutch East India Company,French East India Company,Swedish East India Company,Treaties of the British East India Company,Ships of the British East India Company,Properties of the East India Company,Military history of the British East India Company,Medals of the Honourable East India Company,Bri

### 获取树结构（带page）

In [200]:
f"Category:{targetCategory}"
"Category:Hospitals_in_India"

'Category:Hospitals_in_india'

'Category:Hospitals_in_India'

In [14]:
import urllib
import re
from treelib import Node, Tree
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

def putSubInTree_withPages(tree,current_node,cur_cat,max_level=3,init_level=0):
    for sub_cat in cur_cat.categorymembers.values():
        if sub_cat.ns == 14:
            sub_node = tree.create_node(sub_cat.title.split(":")[1],parent=current_node.identifier)
            if init_level < max_level:
                init_level = init_level + 1
                putSubInTree_withPages(tree,sub_node,sub_cat,max_level,init_level)
        elif sub_cat.ns == 0:
            _ = tree.create_node("[page] "+sub_cat.title,parent=current_node.identifier)
    return None

tree = Tree()
targetCategory = urllib.parse.quote("Hospitals in india")
targetCategory = re.sub("india","India","Economic_history_of_india")
cat = wiki_wiki.page(f"Category:{targetCategory}")
# cat = wiki_wiki.page("Category:Hospitals_in_India")
cat.exists()

currentNode = tree.create_node(cat.title.split(":")[1])
putSubInTree_withPages(tree,currentNode,cat,3)
tree.show()



True

Economic history of India
├── Child labour in India
│   ├── [page] Blood Bricks Campaign
│   ├── [page] Child labour in India
│   ├── [page] Child labour in the diamond industry
│   ├── [page] Children of Mini-Japan
│   ├── [page] Children of the Pyre
│   ├── [page] Om Prakash Gurjar
│   └── [page] Sumangali (child labour)
├── Colonial Indian companies
│   ├── British East India Company
│   │   ├── British East India Company people
│   │   ├── Medals of the Honourable East India Company
│   │   ├── Military history of the British East India Company
│   │   ├── Properties of the East India Company
│   │   ├── Ships of the British East India Company
│   │   ├── Treaties of the British East India Company
│   │   ├── [page] Addiscombe Military Seminary
│   │   ├── [page] Amboyna massacre
│   │   ├── [page] Anchuthengu Fort
│   │   ├── [page] Armagaon
│   │   ├── [page] Batta
│   │   ├── [page] Battle of Buxar
│   │   ├── [page] Battle of Jhelum (1857)
│   │   ├── [page] Battle of Miranpur 

In [35]:
import urllib
import re
import pickle

def putSubInTree(tree,current_node,cur_cat,max_level=3,init_level=0):
    for sub_cat in cur_cat.categorymembers.values():
        if sub_cat.ns == 14:
            sub_node = tree.create_node(sub_cat.title.split(":")[1],parent=current_node.identifier)
            if init_level < max_level:
                init_level = init_level + 1
                putSubInTree(tree,sub_node,sub_cat,max_level,init_level)
    return None

with open("/home/zhoutong/tags/AbstractTags.txt","r+") as f:
    content = [i.strip().split("\t") for i in f.readlines()]

print(f"total: {len(content)}")
content[:3]
# assert False
failList = []
for (target,target_bk) in tqdm_notebook(content[4:]):
    print(f"at {target},{target_bk}")
    tree = Tree()
    cat = wiki_wiki.page(f"Category:{target_bk}")
    if not cat.exists():
        print(f"    bk not exists: {target_bk}")
        cat = wiki_wiki.page(f"Category:{target}")
    if cat.exists():
        path = f'/home/zhoutong/tags/AbstractTags.tree/{target}'
        currentNode = tree.create_node(cat.title.split(":")[1])
        putSubInTree(tree,currentNode,cat,10)
        with open(path,"wb+") as f:
            pickle.dump(tree,f)
#         tree.save2file(f'/home/zhoutong/tags/AbstractTags.tree/{target}')
#         with open(path,"rb+") as f:
#             pickle.load(f).show()
    else:
        print(f"    neither exists: {target} & {target_bk}")
        failList.append([target,target_bk])

total: 343


[['Timeline', 'Timeline_of_Indian_history'],
 ['Astronomy', 'Indian_astronomy'],
 ['Economics', 'Economic_history_of_India']]

at Maritime,Indian_maritime_history
    bk not exists: Indian_maritime_history
    neither exists: Maritime & Indian_maritime_history
at Mathematics,Indian_mathematics
at Metallurgy,History_of_metallurgy_in_the_Indian_subcontinent
    bk not exists: History_of_metallurgy_in_the_Indian_subcontinent
at Military,Military_history_of_India
at Postal,Postage_stamps_and_postal_history_of_India
    bk not exists: Postage_stamps_and_postal_history_of_India
    neither exists: Postal & Postage_stamps_and_postal_history_of_India
at Science_and_technology,History_of_science_and_technology_in_the_Indian_subcontinent
    bk not exists: History_of_science_and_technology_in_the_Indian_subcontinent
at Stone_Age,South_Asian_Stone_Age
    bk not exists: South_Asian_Stone_Age
at Indus_Valley_Civilization,Indus_Valley_Civilization
at Vedic_period,Vedic_period
at Mahajanapadas,Mahajanapada
    bk not exists: Mahajanapada
at Mauryas,Maurya_Empire
at Middle_kingdoms,Middle_kingdoms_of_India
    bk not exists:

    bk not exists: National_Security_Guard
    neither exists: National_Security_Guard_(NSG) & National_Security_Guard
at Railway_Protection_Force_(RPF),Railway_Protection_Force
    bk not exists: Railway_Protection_Force
    neither exists: Railway_Protection_Force_(RPF) & Railway_Protection_Force
at Sashastra_Seema_Bal_(SSB),Sashastra_Seema_Bal
    bk not exists: Sashastra_Seema_Bal
    neither exists: Sashastra_Seema_Bal_(SSB) & Sashastra_Seema_Bal
at Special_Protection_Group_(SPG),Special_Protection_Group
    bk not exists: Special_Protection_Group
    neither exists: Special_Protection_Group_(SPG) & Special_Protection_Group
at Bureau_of_Police_Research_and_Development_(BPR&D),Bureau_of_Police_Research_and_Development
    bk not exists: Bureau_of_Police_Research_and_Development
    neither exists: Bureau_of_Police_Research_and_Development_(BPR&D) & Bureau_of_Police_Research_and_Development
at Central_Bureau_of_Investigation_(CBI),Central_Bureau_of_Investigation
at Directorate_of_Re

    bk not exists: Narcotics_Control_Bureau
    neither exists: Narcotics_Control_Bureau_(NCB) & Narcotics_Control_Bureau
at National_Investigation_Agency_(NIA),National_Investigation_Agency
    bk not exists: National_Investigation_Agency
    neither exists: National_Investigation_Agency_(NIA) & National_Investigation_Agency
at Research_and_Analysis_Wing_(R&AW),Research_and_Analysis_Wing
at Army,Indian_Army
at Navy,Indian_Navy
at Air_Force,Indian_Air_Force
at Censorship,Censorship_in_India
at Elections,Elections_in_India
at Nationalism,Indian_nationalism
at Parliament,Parliament_of_India
at Lok_Sabha,Lok_Sabha
at Rajya_Sabha,Rajya_Sabha
at INC,Indian_National_Congress
at BJP,Bharatiya_Janata_Party
at BSP,Bahujan_Samaj_Party
at CPI,Communist_Party_of_India
at CPM,Communist_Party_of_India_(Marxist)
at NCP,Nationalist_Congress_Party
at Reservations,Reservation_in_India
at Scheduled_groups,Scheduled_Castes_and_Scheduled_Tribes
    bk not exists: Scheduled_Castes_and_Scheduled_Tribes
    n

at Media,Media_of_India
at Cinema,Cinema_of_India
at Television,Television_in_India
at Printing,Printing_industry_in_India
    bk not exists: Printing_industry_in_India
at Mining,Mining_in_India
at Pharmaceuticals,Pharmaceutical_industry_in_India
    bk not exists: Pharmaceutical_industry_in_India
at Retail,Retailing_in_India
at Science_and_technology,Science_and_technology_in_India
at Biotechnology,Biotechnology_in_India
at Telecommunications,Telecommunications_in_India
at Textiles,Textile_industry_in_India
    bk not exists: Textile_industry_in_India
at Tourism,Tourism_in_India
at Transport,Transport_in_India
at Aviation,Aviation_in_India
at Civil,Civil_Aviation_in_India
    bk not exists: Civil_Aviation_in_India
    neither exists: Civil & Civil_Aviation_in_India
at Ports,Ports_in_India
    bk not exists: Ports_in_India
at Rail,Rail_transport_in_India
at Utilities,Category:Utilities_of_India
    bk not exists: Category:Utilities_of_India
    neither exists: Utilities & Category:Util

In [36]:
with open("/home/zhoutong/tags/AbstractTags.tree/Mahajanapadas","rb+") as f:
    tree = pickle.load(f)

In [37]:
tree.show()

Mahajanapadas
├── Avanti (India)
├── Cities and towns in Anga Desh
│   ├── Bhagalpur
│   │   ├── Buildings and structures in Bhagalpur
│   │   │   ├── Schools in Bhagalpur
│   │   │   └── Universities and colleges in Bhagalpur
│   │   │       └── Tilka Manjhi Bhagalpur University
│   │   │           └── Colleges affiliated to Tilka Manjhi Bhagalpur University
│   │   ├── Education in Bhagalpur
│   │   │   ├── Schools in Bhagalpur
│   │   │   └── Universities and colleges in Bhagalpur
│   │   │       └── Tilka Manjhi Bhagalpur University
│   │   │           └── Colleges affiliated to Tilka Manjhi Bhagalpur University
│   │   ├── Media in Bhagalpur
│   │   │   └── Newspapers published in Bhagalpur
│   │   ├── People from Bhagalpur
│   │   └── Transport in Bhagalpur
│   └── Jamalpur, Bihar
├── Gandhara
│   └── Taxila Tehsil
│       └── Populated places in Taxila Tehsil
├── Kosala
│   ├── Kosli language
│   └── People from Kosala
└── Magadha
    ├── Films set in the Magadha Empire
    ├── 

In [24]:
import pandas as pd
import json
from pandas.io.json import json_normalize

with open("/home/zhoutong/tags/tagMap_9k.wikiRes","r+") as f:
    content = [i.strip() for i in f.readlines()][:5]

jsonDictList = [json.loads(i) for i in content]


df = json_normalize(jsonDictList)
df = df[['query','status','categories','links','summary','title','url']]
df.info()
df.head(2)
df['len_categories'] = [len(i) for i in df['categories']]
df.head(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
query         5 non-null object
status        5 non-null object
categories    5 non-null object
links         5 non-null object
summary       5 non-null object
title         5 non-null object
url           5 non-null object
dtypes: object(7)
memory usage: 360.0+ bytes


,query,status,categories,links,summary,title,url
0,Indian Premier League,success,"[2008 establishments in India, All Wikipedia a...","[2008 Champions League Twenty20, 2008 Indian P...",The Indian Premier League (IPL) is a professio...,Indian Premier League,https://en.wikipedia.org/wiki/Indian_Premier_L...
1,Bharat Financial Inclusion,success,"[1998 establishments in India, All Wikipedia a...","[Adverse selection, Andhra Pradesh, Asymmetric...",Bharat Financial Inclusion Limited (formerly k...,Bharat Financial Inclusion,https://en.wikipedia.org/wiki/Bharat_Financial...


,query,status,categories,links,summary,title,url,len_categories
0,Indian Premier League,success,"[2008 establishments in India, All Wikipedia a...","[2008 Champions League Twenty20, 2008 Indian P...",The Indian Premier League (IPL) is a professio...,Indian Premier League,https://en.wikipedia.org/wiki/Indian_Premier_L...,19
1,Bharat Financial Inclusion,success,"[1998 establishments in India, All Wikipedia a...","[Adverse selection, Andhra Pradesh, Asymmetric...",Bharat Financial Inclusion Limited (formerly k...,Bharat Financial Inclusion,https://en.wikipedia.org/wiki/Bharat_Financial...,10


# 【工具封装】WikiSearch

In [89]:
import wikipediaapi

class WikiSearch():
    def __init__(self):
        self.wiki_wiki = wikipediaapi.Wikipedia('en')
        
    def search(self,targetWord= "BJP"):
        page_py = self.wiki_wiki.page(targetWord)
        if page_py.exists():
            title = page_py.title
            url = page_py.canonicalurl
            cat = [i.title.split("Category:")[1] for i in page_py.categories.values()]
            summary = page_py.summary
            links = [i.title for i in page_py.links.values() if ":" not in i.title]
            status="success"
        else:
            title = ""
            url = ""
            cat = []
            summary = ""
            links = []
            status="fail"
            print("page not exists")
        return (title,url,cat,summary,links,status)
    def getResult_json(self,query,verbose=False):
        (title,url,categories,summary,links,status) = self.search(query)
        return json.dumps({"query":query,"title":title,"url":url,"categories":categories,"summary":summary,"links":links,"status":status})
        

In [91]:
w_searcher = WikiSearch()
print(w_searcher.getResult_json("Dream"))

{"query": "Dream", "title": "Dream", "url": "https://en.wikipedia.org/wiki/Dream", "categories": ["All articles with dead external links", "All articles with failed verification", "All articles with unsourced statements", "Articles to be expanded from February 2019", "Articles with Curlie links", "Articles with dead external links from April 2013", "Articles with evidence out of context", "Articles with failed verification from April 2015", "Articles with unsourced statements from July 2016", "CS1 maint: Archived copy as title", "CS1 maint: Multiple names: authors list", "Commons category link is locally defined", "Dream", "Night", "Psychotherapy", "Sleep", "Symbols", "Webarchive template wayback links", "Wikipedia articles needing clarification from January 2019", "Wikipedia articles with GND identifiers", "Wikipedia articles with LCCN identifiers", "Wikipedia articles with NDL identifiers", "Wikipedia pages semi-protected against vandalism"], "summary": "A dream is a succession of im

In [98]:
page_cat = list(page_py.categories.values())[0]
type(page_cat)
type(page_py)
page_cat.title
page_py.ns
page_cat.ns
wikipediaapi.Namespace.CATEGORY
page_cat.links

wikipediaapi.WikipediaPage

wikipediaapi.WikipediaPage

'Category:All Wikipedia articles written in American English'

0

14

<Namespace.CATEGORY: 14>

{'Wikipedia:Categorization': Wikipedia:Categorization (id: ??, ns: 4),
 'Wikipedia:Maintenance': Wikipedia:Maintenance (id: ??, ns: 4),
 'Wikipedia:What is an article?': Wikipedia:What is an article? (id: ??, ns: 4),
 'Help:Category': Help:Category (id: ??, ns: 12),
 'Category:Hidden categories': Category:Hidden categories (id: ??, ns: 14)}

# 【方案二】纯爬虫处理

发现不论哪家的API都会有一些信息的滞后，和当前wiki页面的情况不符<br>
诸如新的重定向没有生效、 category取的不对

In [ ]:
import requests
word = "ADVICE"
def ws(word):
    res = requests.get(f"https://en.wikipedia.org/w/index.php?search={word}&title=Special%3ASearch&go=Go")
    with open("/Users/zac/Downloads/currentPage.html","w+") as f:
        f.writelines(res.text)
    if(not res.url.startswith("https://en.wikipedia.org/wiki/")):
        print("error")
    return res
    

## 【工具封装】WikiSearch_byCrawl

In [106]:
import requests
import json
from lxml import etree
class WikiSearch_byCrawl():
    def __init__(self,verbosePath="wikiSearchResult_request.html"):
        self.verbosePath = verbosePath
        self.url = "https://en.wikipedia.org/w/index.php" 
        self.gl_proxies = {}
        self.header = {"Referer":"https://www.google.com/",
                       'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}
        pass
    
    def request(self,query,verbose=False):
        params = {'search':query,
                  'title':'Special%3ASearch',
                  'go':'Go'}
        resp1 = requests.get(self.url,params=params,headers=self.header,proxies=self.gl_proxies,timeout=4)
        if(verbose):
            with open(self.verbosePath,"w+") as f: f.writelines(resp1.text)
        return resp1
    
    @staticmethod
    def _findCategory(html):
        cats = html.xpath("//div[@id='mw-normal-catlinks']/ul/li")
        allCategories = [li.xpath("string(.)") for li in cats]
        return allCategories
    @staticmethod
    def _findHiddenCategory(html):
        cats = html.xpath("//div[@id='mw-hidden-catlinks']/ul/li")
        allCategories = [li.xpath("string(.)") for li in cats]
        return allCategories
    @staticmethod
    def _findAllLinks(html):
        content_div = html.xpath("//div[@id='mw-content-text']")[0]
        allLink = [a for a in content_div.xpath(".//a[@href and @title]")]
        allLink = [a for a in allLink if a.attrib['href'].startswith("/wiki/")] # 过滤掉非wiki词条的
        allLink = [a for a in allLink if ":" not in a.attrib['href']] # 过滤掉非wiki词条的
        allLinkRes = [a.xpath("string(.)") for a in allLink]
        return allLinkRes
    # todo
    @staticmethod
    def _findSummary(html):
        content_text_div = html.xpath("//div[@id='mw-content-text']/div[@class='mw-parser-output']")[0]
        allText = content_text_div.xpath("./p[not(@*)]") #全部段落
        return allText[0].xpath("string(.)")
    
    def parse(self,response):
        try:
            html = etree.HTML(response.text)
            cats = self._findCategory(html)
            hidden_cats = self._findHiddenCategory(html)
            links = self._findAllLinks(html)
            summary = self._findSummary(html)
            title = html.xpath("//h1[@id='firstHeading']")[0].xpath("string(.)")
            url = response.url
            resDict = {"title":title,"summary":summary,"categories":cats,"hidden_categories":hidden_cats,"links":links,"url":url}
            resDict.update({"status":"success"})
        except Exception as e:
            resDict = {"title":"","summary":"","categories":[],"hidden_categories":[],"links":[],"url":""}
            resDict.update({"status":"fail"})
        return resDict
        
    
    def search(self,query,verbose=False):
        response = self.request(query,verbose)
        return self.parse(response)
    
    def getResult_json(self,query,verbose=False):
        resDict = self.search(query,verbose)
        resDict.update({"query":query})
        resultJSON=json.dumps(resDict)
        return resultJSON

In [93]:
query = "ADVICE"
wsearcher_crawl = WikiSearch_byCrawl()
response = wsearcher_crawl.request(query,True)
print(wsearcher_crawl.getResult_json(query))

{"title": "Advice", "summary": "Advice (noun) or advise (verb) may refer to:\n", "categories": ["Disambiguation pages"], "hidden_categories": ["Disambiguation pages with short description", "All article disambiguation pages", "All disambiguation pages"], "links": ["Advice (opinion)", "Advice (constitutional)", "Advice (programming)", "Advice (complexity)", "pay slip", "HMS\u00a0Advice", "\"Advice\" (song)", "Christina Grimmie", "Kehlani", "SweetSexySavage", "ADVISE", "The Advice", "The Advice (album)", "Advisor", "Advocate"], "url": "https://en.wikipedia.org/wiki/Advice"}


# 两个工具类对比
- 时间和准确性上都是 byCrawl 更优

In [103]:
query = "China"

In [104]:
w_searcher = WikiSearch()
print(">>> WikiSearch()\n",w_searcher.getResult_json(query))

>>> WikiSearch()
 {"query": "China", "title": "China", "url": "https://en.wikipedia.org/wiki/China", "categories": ["All Wikipedia articles written in American English", "All articles containing potentially dated statements", "All articles with vague or ambiguous time", "Articles containing Chinese-language text", "Articles containing Mongolian-language text", "Articles containing Persian-language text", "Articles containing Portuguese-language text", "Articles containing Sanskrit-language text", "Articles containing potentially dated statements from 2005", "Articles containing potentially dated statements from 2010", "Articles containing potentially dated statements from 2012", "Articles containing potentially dated statements from 2013", "Articles containing potentially dated statements from 2015", "Articles containing potentially dated statements from 2017", "Articles containing potentially dated statements from 2018", "Articles containing potentially dated statements from January 2

In [107]:
wsearcher_crawl = WikiSearch_byCrawl()
print(">>> WikiSearch_byCrawl()\n",wsearcher_crawl.getResult_json(query))

>>> WikiSearch_byCrawl()
 {"title": "China", "summary": "China, officially the People's Republic of China (PRC), is a country in East Asia and the world's most populous country, with a population of around 1.404\u00a0billion.[10] Covering approximately 9,600,000 square kilometers (3,700,000\u00a0sq\u00a0mi), it is the third- or fourth-largest country by total area.[k][16] Governed by the Communist Party of China, the state exercises jurisdiction over 22 provinces, five autonomous regions, four direct-controlled municipalities (Beijing, Tianjin, Shanghai, and Chongqing), and the special administrative regions of Hong Kong and Macau.\n", "categories": ["States and territories established in 1949", "China", "Atheist states", "BRICS nations", "East Asian countries", "G20 nations", "Member states of the United Nations", "Northeast Asian countries"], "hidden_categories": ["Pages using Timeline", "Articles containing Portuguese-language text", "Articles containing Chinese-language text", "Art